In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from wx_hyperparam import WxHyperParameter
from wx_core import wx_slp, wx_mlp, connection_weight

In [2]:
def get_sample_data(num_cls=2):
    train_num = 100
    test_num = 100
    input_dim = 20000
    num_cls = num_cls
    if num_cls < 2:
        return

    x_train = np.random.random((train_num, input_dim))
    y_train = to_categorical(np.random.randint(num_cls, size=(train_num, 1)), num_classes=num_cls)

    x_test = np.random.random((test_num, input_dim))
    y_test = to_categorical(np.random.randint(num_cls, size=(test_num, 1)), num_classes=num_cls)

    return x_train, y_train, x_test, y_test


def StandByCol(data_frame, unused_cols=[]):
    unused_cols_list = []
    for col_ in unused_cols:
        unused_cols_list.append(data_frame[col_])
    data_frame = data_frame.drop(unused_cols,axis=1)

    data_frame = data_frame.astype(float)
    data_frame.fillna(0, inplace=True)
    data_frame = data_frame.apply(lambda x: ((x-x.mean())/x.std()), axis=0)

    for n,col_ in enumerate(unused_cols):
        data_frame[col_] = unused_cols_list[n]

    return data_frame

In [3]:
#read from actual datas
df_data = pd.read_csv('input_data_1.csv')
f_names = df_data['fnames'].values
df_data = df_data.drop(['fnames'],axis=1)
id_names = df_data.columns.values

In [4]:
#z-scoring
df_data = StandByCol(df_data,unused_cols=[])
df_data

,Sample0,Sample1,Sample2,Sample3,Sample4,Sample5,Sample6,Sample7,Sample8,Sample9,Sample10,Sample11,Sample12,Sample13,Sample14,Sample15,Sample16,Sample17,Sample18,Sample19
0,-0.231762,-0.247491,-0.248976,-0.202346,-0.186685,-0.223989,-0.179934,-0.214987,-0.166885,-0.280402,-0.089822,-0.124037,-0.114895,-0.098585,-0.166281,-0.111795,-0.179398,-0.124769,-0.115809,-0.115490
1,-0.238106,-0.284280,-0.262749,-0.237539,-0.216971,-0.231590,-0.217126,-0.219176,-0.167156,-0.327269,-0.113020,-0.158831,-0.173408,-0.107497,-0.181686,-0.128897,-0.193744,-0.130680,-0.128032,-0.127016
2,-0.237680,-0.283885,-0.262749,-0.237539,-0.216971,-0.240506,-0.195062,-0.219176,-0.170003,-0.327269,-0.112947,-0.158831,-0.173323,-0.107497,-0.181631,-0.128968,-0.193653,-0.130680,-0.127868,-0.127016
3,-0.205081,-0.257032,-0.221823,-0.210507,-0.196147,-0.212986,-0.202461,-0.184733,-0.146118,-0.242611,-0.104094,-0.150289,-0.155821,-0.093080,-0.171468,-0.120538,-0.180182,-0.123729,-0.118418,-0.117917
4,2.161295,7.610784,4.325499,5.389036,11.271672,1.678158,2.977458,0.659782,4.251222,1.553357,5.939371,4.620631,17.209243,5.795145,23.720365,8.329439,25.973607,15.287847,13.258032,8.344457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20497,-0.237936,-0.278151,-0.255565,-0.221896,-0.215083,-0.222839,-0.215396,-0.216464,-0.164038,-0.234033,-0.113020,-0.158623,-0.173323,-0.107497,-0.181631,-0.128616,-0.193653,-0.130680,-0.128032,-0.126883
20498,0.027078,-0.042573,-0.134075,-0.018481,-0.031766,-0.108912,-0.077528,-0.168777,-0.056130,-0.021716,-0.021378,-0.047679,-0.033593,-0.027739,-0.004701,-0.026959,-0.007284,-0.036701,-0.008642,-0.041510
20499,0.453567,0.437581,1.209685,1.395503,0.577594,0.390636,0.969745,0.724169,0.130132,1.101128,0.687686,1.011803,1.250106,0.438208,2.004382,0.868224,1.729439,1.635334,0.923658,1.121414
20500,-0.025755,0.065084,0.054222,0.044495,0.196145,-0.073413,-0.040898,-0.130096,0.093666,-0.143986,-0.025048,0.000924,-0.085424,-0.005774,0.108933,0.086321,0.145991,0.056075,0.068434,0.100185


In [5]:
df_data = df_data.T
x_all = df_data.values  
df_data

,0,1,2,3,4,5,6,7,8,9,...,20492,20493,20494,20495,20496,20497,20498,20499,20500,20501
Sample0,-0.231762,-0.238106,-0.237680,-0.205081,2.161295,-0.238021,-0.206202,-0.237850,-0.238106,-0.079438,...,-0.184203,-0.111937,-0.228407,-0.166812,-0.004315,-0.237936,0.027078,0.453567,-0.025755,-0.097900
Sample1,-0.247491,-0.284280,-0.283885,-0.257032,7.610784,-0.283885,-0.106731,-0.284181,-0.284280,-0.098823,...,-0.223402,-0.196099,-0.265300,-0.132830,0.077738,-0.278151,-0.042573,0.437581,0.065084,-0.072428
Sample2,-0.248976,-0.262749,-0.262749,-0.221823,4.325499,-0.262749,-0.166221,-0.262479,-0.262390,-0.103365,...,-0.121692,0.020729,-0.246496,-0.163796,-0.034224,-0.255565,-0.134075,1.209685,0.054222,-0.118451
Sample3,-0.202346,-0.237539,-0.237539,-0.210507,5.389036,-0.236385,-0.103795,-0.237020,-0.237193,-0.090807,...,-0.133908,-0.055885,-0.227784,-0.153899,-0.009187,-0.221896,-0.018481,1.395503,0.044495,-0.077416
Sample4,-0.186685,-0.216971,-0.216971,-0.196147,11.271672,-0.216971,-0.180656,-0.216535,-0.216971,-0.058639,...,-0.155550,-0.180366,-0.197942,-0.083043,0.097078,-0.215083,-0.031766,0.577594,0.196145,-0.044985
Sample5,-0.223989,-0.231590,-0.240506,-0.212986,1.678158,-0.231508,-0.142264,-0.239928,-0.240424,-0.064497,...,-0.138837,-0.021732,-0.220941,-0.086622,0.008318,-0.222839,-0.108912,0.390636,-0.073413,-0.055250
Sample6,-0.179934,-0.217126,-0.195062,-0.202461,2.977458,-0.216982,-0.068587,-0.216982,-0.215828,-0.056906,...,-0.143433,0.109083,-0.206166,-0.056041,-0.039889,-0.215396,-0.077528,0.969745,-0.040898,-0.106227
Sample7,-0.214987,-0.219176,-0.219176,-0.184733,0.659782,-0.144678,-0.100471,-0.219073,-0.219176,-0.047668,...,-0.150555,-0.089982,-0.216106,-0.181722,-0.076986,-0.216464,-0.168777,0.724169,-0.130096,-0.066906
Sample8,-0.166885,-0.167156,-0.170003,-0.146118,4.251222,-0.169867,-0.132317,-0.169867,-0.169867,-0.063451,...,-0.076218,-0.101544,-0.155091,-0.057622,0.114271,-0.164038,-0.056130,0.130132,0.093666,-0.011937
Sample9,-0.280402,-0.327269,-0.327269,-0.242611,1.553357,-0.320154,-0.015009,-0.325306,-0.327269,-0.002087,...,-0.032596,0.130652,-0.305841,-0.196902,0.112169,-0.234033,-0.021716,1.101128,-0.143986,-0.036192


In [6]:
df_anno = pd.read_csv('input_data_2.csv')
anno_ids = df_anno.id.values
anno_class = df_anno.label.values
class_type = df_anno.label.unique()

In [7]:
anno_ids

array(['Sample0', 'Sample1', 'Sample2', 'Sample3', 'Sample4', 'Sample5',
       'Sample6', 'Sample7', 'Sample8', 'Sample9', 'Sample10', 'Sample11',
       'Sample12', 'Sample13', 'Sample14', 'Sample15', 'Sample16',
       'Sample17', 'Sample18', 'Sample19'], dtype=object)

In [8]:
anno_class

array(['Tumor', 'Tumor', 'Tumor', 'Tumor', 'Tumor', 'Tumor', 'Tumor',
       'Tumor', 'Tumor', 'Tumor', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [9]:
class_type

array(['Tumor', 'Normal'], dtype=object)

In [10]:
n_cls = len(class_type)
y_all = []
for id_ in id_names:
    idx = np.where(anno_ids == id_)
    y_all.append(np.where(class_type == anno_class[idx])[0][0])
y_all = np.asarray(y_all)
y_all = to_categorical(y_all, num_classes=n_cls)

y_all 

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [11]:
#split to train and val
x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, test_size=0.2, random_state=1)

In [12]:
x_train

array([[-2.48976301e-01, -2.62748842e-01, -2.62748842e-01, ...,
         1.20968516e+00,  5.42222426e-02, -1.18450672e-01],
       [-1.66281463e-01, -1.81686327e-01, -1.81631186e-01, ...,
         2.00438175e+00,  1.08932969e-01, -8.54748492e-02],
       [-1.86684604e-01, -2.16971028e-01, -2.16971028e-01, ...,
         5.77593876e-01,  1.96144627e-01, -4.49847619e-02],
       ...,
       [-1.14894803e-01, -1.73408296e-01, -1.73322625e-01, ...,
         1.25010571e+00, -8.54238940e-02, -9.48477255e-02],
       [-1.24037335e-01, -1.58831134e-01, -1.58831134e-01, ...,
         1.01180311e+00,  9.23805117e-04, -1.00705319e-01],
       [-2.23988538e-01, -2.31590305e-01, -2.40506371e-01, ...,
         3.90635687e-01, -7.34126509e-02, -5.52502900e-02]])

In [13]:
y_train

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [14]:
hp = WxHyperParameter(epochs=30, learning_ratio=0.01, batch_size=8, verbose=False)

In [15]:
sel_idx, sel_weight, val_acc = wx_slp(x_train, y_train, x_val, y_val, n_selection=10, hyper_param=hp, num_cls=n_cls)

Epoch 1/30
1/2 [==============>...............] - ETA: 0s - loss: 1.0491 - accuracy: 0.6250
Epoch 00001: val_loss improved from inf to 1.79270, saving model to ./slp_wx_weights_best.hdf5
2/2 [==============================] - 0s 140ms/step - loss: 1.6983 - accuracy: 0.5000 - val_loss: 1.7927 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/30
1/2 [==============>...............] - ETA: 0s - loss: 2.7792 - accuracy: 0.5000
Epoch 00002: val_loss improved from 1.79270 to 0.00000, saving model to ./slp_wx_weights_best.hdf5
2/2 [==============================] - 0s 25ms/step - loss: 1.4275 - accuracy: 0.7500 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/30
1/2 [==============>...............] - ETA: 0s - loss: 1.1332e-04 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 12ms/step - loss: 5.6658e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/30
1/2 [==============>........

1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00028: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 29/30
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00029: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 30/30
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00030: val_loss did not improve from 0.00000
1/1 [==============================] - 0s 767us/step - loss: 0.0000e+00 - accuracy: 1.0000

function [wx_slp] finished in 2.8156471252441406 s


In [16]:
print ('\nSingle Layer WX')
print ('selected feature names:',f_names[sel_idx])
print ('selected feature index:',sel_idx)
print ('selected feature weight:',sel_weight)
print ('evaluation accuracy:',val_acc)


Single Layer WX
selected feature names: ['SFTPC' 'SFTPA2' 'SFTPA1' 'FTL' 'EEF1A1' 'ACTB' 'GAPDH' 'ACTG1' 'HLA-B'
 'FGG']
selected feature index: [15871 15869 15868  6668  5350   200  6827   203  7870  6380]
selected feature weight: [27.63028513 18.62849216  5.74430402  3.84302923  2.83739353  2.40131759
  1.24756945  1.1751258   0.95742483  0.81372882]
evaluation accuracy: 1.0


In [18]:
all_genes = len(f_names)

20502

In [24]:
sel_idx, sel_weight, val_acc = wx_slp(x_train, y_train, x_val, y_val, n_selection=len(f_names), hyper_param=hp, num_cls=n_cls)

Epoch 1/30
1/2 [==============>...............] - ETA: 0s - loss: 0.4235 - accuracy: 0.7500
Epoch 00001: val_loss improved from inf to 2.24856, saving model to ./slp_wx_weights_best.hdf5
2/2 [==============================] - 0s 39ms/step - loss: 1.1801 - accuracy: 0.5625 - val_loss: 2.2486 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/30
1/2 [==============>...............] - ETA: 0s - loss: 2.2524 - accuracy: 0.6250
Epoch 00002: val_loss improved from 2.24856 to 0.00000, saving model to ./slp_wx_weights_best.hdf5
2/2 [==============================] - 0s 15ms/step - loss: 2.0208 - accuracy: 0.6875 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/30
1/2 [==============>...............] - ETA: 0s - loss: 3.0916e-04 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 12ms/step - loss: 1.5458e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/30
1/2 [==============>.........

1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00028: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 29/30
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00029: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 30/30
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00030: val_loss did not improve from 0.00000
1/1 [==============================] - 0s 689us/step - loss: 0.0000e+00 - accuracy: 1.0000

function [wx_slp] finished in 1.2418487071990967 s


In [25]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

In [26]:

def load_RDS(file_path):

    read_RDS = robjects.r['readRDS']
    return read_RDS(file_path)



def load_csv(file_path):
    return pd.read_csv(file_path, index_col=0)



def pandas_to_r(df):

    with localconverter(robjects.default_converter + pandas2ri.converter):
        r_from_pandas_df = robjects.conversion.py2rpy(df)
    return r_from_pandas_df



def r_to_pandas(df):
    with localconverter(robjects.default_converter + pandas2ri.converter):
            pandas_from_r_df = robjects.conversion.rpy2py(df)
    return pandas_from_r_df

In [27]:
main = r_to_pandas(load_RDS("/home/colombelli/Documents/datasets/research/brca.rds"))

In [35]:
df = main[0:200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [39]:
anno_class = df['class'].values
class_type = df['class'].unique()

In [41]:
anno_class

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [46]:
def to_categorical(anno_class):
    
    anno_y = []
    for cls in anno_class:
        new_y = [0, 0]
        new_y[int(cls)] = 1
        anno_y.append(new_y)
        
    return np.array(anno_y, dtype=float)

In [51]:
y_true = to_categorical(anno_class)

In [59]:
df.iloc[:, 0:-1]

,SCGB2A2,SCGB1D2,MUCL1,TFF1,PIP,GSTM1,CYP2B7P1,CPB1,TFAP2B,LTF,...,PPIG,LOC100125556,EXOC4,TEC,TUSC2,MYOM3,TRIT1,OTUD7A,USP27X,ARPM1
TCGA.A7.A0CE.11A.21R.A089.07,10.554105,9.403274,11.540931,10.885613,12.365593,7.116029,4.282285,8.959008,4.681972,10.166929,...,5.796821,2.920358,5.514571,1.463288,5.777505,2.904490,4.777675,1.081973,3.198914,2.086866
TCGA.A7.A0CH.11A.32R.A089.07,12.073295,10.504623,13.597325,5.261387,13.478214,7.906778,4.471297,6.055256,4.302780,9.480826,...,4.963855,2.261438,5.410830,1.598669,5.715822,0.936055,4.207983,1.269942,2.587507,1.942952
TCGA.A7.A0D9.11A.53R.A089.07,4.652332,3.530492,1.508318,0.426305,1.940593,7.007359,1.914802,3.613250,0.692070,5.399014,...,5.841457,1.566242,6.175997,1.566785,5.381756,0.291109,3.960901,1.176532,2.412553,2.647025
TCGA.A7.A0DB.11A.33R.A089.07,1.761845,1.067840,2.404918,0.000000,0.502656,6.964555,1.093856,3.814888,0.014930,4.048145,...,5.429606,1.782776,5.905556,2.404466,5.122230,0.423852,3.972255,1.239671,2.236906,2.082817
TCGA.A7.A0DC.11A.41R.A089.07,10.252504,8.455516,10.585970,4.566540,8.184018,6.626465,3.178531,10.345035,3.593233,10.683944,...,5.811546,2.560664,6.023154,1.526697,5.571172,1.779448,4.574964,1.429833,2.755122,2.044148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.A2.A0YF.01A.21R.A109.07,5.038334,1.253711,3.693359,12.327187,10.168611,8.107207,5.954676,6.593731,0.306234,9.413047,...,6.534514,3.496705,6.282955,1.672766,6.086834,0.523894,4.025191,0.320993,3.962603,3.188679
TCGA.A2.A0YG.01A.21R.A109.07,8.100245,3.338998,9.162506,7.940139,9.660221,2.253116,6.509444,9.613153,0.904108,8.791196,...,6.433270,3.134508,6.636783,1.467468,5.966175,0.706581,4.452151,0.623535,1.953326,1.895817
TCGA.A2.A0YH.01A.11R.A109.07,6.922006,5.330330,5.801106,9.149674,4.247552,3.544134,8.185333,8.434137,0.088184,5.861760,...,5.410977,3.980932,6.022863,1.552861,6.021822,0.525002,4.555630,0.256139,3.301557,1.914174
TCGA.A2.A0YI.01A.31R.A10J.07,8.072254,6.823353,3.746149,9.944818,9.986013,7.478888,7.067757,15.659412,1.628264,13.565433,...,6.234599,2.542349,5.627656,1.630827,5.738051,0.889373,5.179193,0.533942,3.761526,2.004443


In [65]:
x_all = df.iloc[:, 0:-1].values
x_all

array([[10.55410472,  9.40327405, 11.54093124, ...,  1.08197275,
         3.1989141 ,  2.08686602],
       [12.07329462, 10.50462343, 13.59732499, ...,  1.26994219,
         2.58750748,  1.94295236],
       [ 4.65233205,  3.53049232,  1.5083182 , ...,  1.17653231,
         2.41255306,  2.64702489],
       ...,
       [ 6.92200564,  5.33032974,  5.80110554, ...,  0.25613937,
         3.30155738,  1.91417361],
       [ 8.07225401,  6.82335279,  3.74614941, ...,  0.53394192,
         3.76152636,  2.00444257],
       [ 1.5215571 ,  0.44084534,  0.14627491, ...,  0.22839752,
         2.60974696,  1.83957963]])

In [75]:
features = list(df.columns)

In [90]:
sel_idx, sel_weight, val_acc = wx_slp(x_all, y_true, x_all, y_true, n_selection=len(features), hyper_param=hp, num_cls=2)

Epoch 1/30
 1/25 [>.............................] - ETA: 0s - loss: 4.1003 - accuracy: 0.3750
Epoch 00001: val_loss improved from inf to 0.37606, saving model to ./slp_wx_weights_best.hdf5
25/25 [==============================] - 0s 5ms/step - loss: 58.1593 - accuracy: 0.7650 - val_loss: 0.3761 - val_accuracy: 0.9850 - lr: 0.0010
Epoch 2/30
 1/25 [>.............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00002: val_loss did not improve from 0.37606
25/25 [==============================] - 0s 3ms/step - loss: 2.9089 - accuracy: 0.9450 - val_loss: 2.1261 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 3/30
 1/25 [>.............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.37606
25/25 [==============================] - 0s 3ms/step - loss: 1.4712 - accuracy: 0.9900 - val_loss: 1.7333 - val_accuracy: 0.9800 - lr: 0.0010
Epoch 4/30
 1/25 [>.............................] - ETA: 0s - loss: 1.5076 - accuracy:

Epoch 28/30
 1/25 [>.............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00028: val_loss did not improve from 0.00000
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 29/30
 1/25 [>.............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00029: val_loss did not improve from 0.00000
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 30/30
 1/25 [>.............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00030: val_loss did not improve from 0.00000
7/7 [==============================] - 0s 923us/step - loss: 0.0000e+00 - accuracy: 1.0000

function [wx_slp] finished in 3.012053966522217 s


In [95]:
def build_rank(df, sel_idx):
    
    print("Processing data...")
    genes = list(df.columns)
    data = {}
    data['gene'] = []
    data['rank'] = []
    for i, gen_id in enumerate(sel_idx):
        data['gene'].append(genes[gen_id])
        data['rank'].append(i+1)

    rank = pd.DataFrame(data, columns=['rank']).set_index(pd.Index(data['gene']))
    return rank

In [97]:
build_rank(df, sel_idx)[0:15]

Processing data...


,rank
FN1,1
MMP11,2
CRYAB,3
COL1A1,4
BGN,5
FABP4,6
SAA1,7
KRT14,8
SFRP1,9
MIR4508,10
